# CNN

Convolutional neural network(CNN) is well-known deep learning technique to solve image classification problems. 

Hence, we will try to solve quick draw tasks using CNN. 

# 1. Data Preprocessing 

## 1.1 Basic Data Loading

Our group will use 5 classes: 'calendar', 'snowman', 'penguin', 'blackberry', 'teddy-bear'. 

Each class have exactly 128153 data. Basic datasets are already processed by **Liu Chang**. 

In [15]:
from read_data import *

In [16]:
dataset_path='/home/di0002ya/quickdraw/dsy'
train_X,train_Y,test_X,test_Y=get_dataset(dataset_path,test_r=0.5)

## 1.2 Basic Data Analysis


In [17]:
print('training image datasets has', len(train_X))
print('testing image datasets has', len(test_X))

training image datasets has 512612
testing image datasets has 64076


Basic datasets are made up of multiple strokes. 

Each strokes are represented by X (dimension 0) and Y (dimension 1) lists. 

Lenght of X or Y reprensents the length of each stroke. 

In [18]:
print('Let us using 1st train sample as example to investigate the structure of data')
print('Number of strokes of 1st sketch is', len(train_X[0]))
print('1st strokes has', len(train_X[0][0][0]), 'points')

Let us using 1st train sample as example to investigate the structure of data
Number of strokes of 1st sketch is 5
1st strokes has 24 points


## 1.3 CNN Data Preprocessing

Normally, RGB images will have 3 channels. Videa will have 4 channels.

In our case, we only deal with grey scale sketch which means that channel will be 1 in our case. 

In order to preprocess sequence stroke data to image data, we will draw each skeches on 42\*42\*1 arrays. The value of each existing points is 1. 

In [19]:
import pandas as pd
df_trainX = pd.DataFrame({'drawing':train_X})
df_testX = pd.DataFrame({'drawing':test_X})

In [20]:
# Calculate Stroke Number of Each Image
df_trainX['stroke_number'] = df_trainX['drawing'].str.len()
df_testX['stroke_number'] = df_testX['drawing'].str.len()

We should do normalization in order to make training less sensitive to the scale of the features. Moreover, it will make optimization well-conditioned, improving the convergence rate of gradient descent. 

In [21]:
# Normalization 
def _array_normalizer(array1,Xmin,Xmax,array_min):
    '''
    function:
        - normalize X,Y array by range of X
        - used in feature_eng_pt2
    input:
        array1 = array that you want to normalize (1D array or list)
        Xmin = minimum value of your X array (int)
        Xmax = maximum value of your X array (int)
        array_min = minimum value of array1

    output:
        normalized array of array1
    '''
    return (np.array(array1)-np.array([array_min]*len(array1)))/float(Xmax-Xmin)

In [22]:
import numpy as np
def preprocess(df):
    X = {} # Dimension 0: storing existing points for each images
    Y = {} # Dimension 1: storing existing points for each images
    Ymax = {} # Dimension 1: storing Ymax for each images
    image_pile = np.zeros((df.shape[0],42*42*1)) # Image Array storing final image info
    orig_index = df.index
    for i in range(df.shape[0]):
        num = df.loc[i,'stroke_number']
        #store X,Y of the stroke in a temp list
        Xt = [df.loc[i,'drawing'][stroke][0] for stroke in range(num)]
        Yt = [df.loc[i,'drawing'][stroke][1] for stroke in range(num)]
        #Flatten Xt & Yt
        Xtemp = [item for stroke in Xt for item in stroke]
        Ytemp = [item for stroke in Yt for item in stroke]
        # Normalize Xtemp&Ytemp for ith drawing
        Xmintemp = np.min(Xtemp)-10
        Xmaxtemp = np.max(Xtemp)+10
        Ymintemp = np.min(Ytemp)-10
        Xnorm = _array_normalizer(Xtemp, Xmintemp,Xmaxtemp,Xmintemp)
        Ynorm = _array_normalizer(Ytemp, Xmintemp,Xmaxtemp,Ymintemp)
        Ymax[i] = np.max(Ynorm)
        X[i] = Xnorm
        Y[i] = Ynorm
        # Draw existing points on the image with corresponding positions
        image = np.zeros((42,42))
        xarray = np.around(np.array(Xnorm)*42)
        yarray = np.around(np.array(Ynorm)*42/float(Ymax[i]))
        xarray[xarray>=42.] = 41
        yarray[yarray>=42.] = 41

        for item in range(len(xarray)):
            image[int(np.around(yarray[item])),int(np.around(xarray[item]))] = 1
        image_pile[i] = image.reshape(1,42*42*1)

    # create new features
    df['Ymax'] = pd.Series(Ymax)
    df['X'] = pd.Series(X)
    df['Y'] = pd.Series(Y)

    #return pd.DataFrame(image_pile, index = orig_index)
    df_final = pd.DataFrame(image_pile, index = orig_index)
    return df_final

In [23]:
trainX = preprocess(df_trainX)
testX = preprocess(df_testX)

In [24]:
#Convert to Array & Normalization 
trainX_ar = np.array(trainX)
testX_ar = np.array(testX)

In [27]:
trainY_ar = np.array(train_Y)
testY_ar = np.array(test_Y)

In [39]:
# Normalization X values 
data_np = np.array(trainX_ar)
data_key = data_np[:,-1]
trainX = data_np[:,0:42*42]
trainX /= 10000 
trainX += 1 
trainX[trainX == 1.0] = 0

In [40]:
trainX = trainX.reshape(len(trainX),42,42,1)

In [42]:
from keras.utils import np_utils
trainY_label = np_utils.to_categorical(trainY_ar,5)
testY_label = np_utils.to_categorical(testY_ar,5)

In [48]:
#Save Data
np.save('/home/di0002ya/quickdraw/dsy/trainX.npy', trainX)
np.save('/home/di0002ya/quickdraw/dsy/testX_ar.npy', testX_ar)
np.save('/home/di0002ya/quickdraw/dsy/trainY_label.npy', trainY_label)
np.save('/home/di0002ya/quickdraw/dsy/testY_label.npy', testY_label)

# 2. Training Model 

## 2.1 Simple Model Structure Design

We firstly build a simple CNN model. Moreover, dropout layer, maxpooling layer will also be used. Order of these layers will be based on the following rules. 
1. Dropout 
2. Linear/Dense/Conv
3. Batch Normalization
4. Activations
5. Maxpooling

'Relu' will be used after convolutional layer and dense layer. 

Output activation function will be 'softmax' as this is a classification problems. 

Adam is the selected optimizer as Adam is the best optimizer currently. 

Loss function will be 'cross-entropy'. 

In [34]:
#Declare 
kernelS = 5 #Kernel size
poolS = 2 #Pool size
dropout_rate = .20 # Dropout rate 
batch = 512 # Batch size
epochs = 10 # Epochs
len_category = 5 # number of classes 
num_filters = 64
dense_neuron = 100

In [35]:
import pandas as pd
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.models import load_model
import tensorflow as tf

In [36]:
# Set GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1',
 '/job:localhost/replica:0/task:0/device:GPU:2',
 '/job:localhost/replica:0/task:0/device:GPU:3']

In [38]:
# Set GPU
import keras
config = tf.ConfigProto( device_count = {'GPU': 3 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [43]:
# Model Building
model = Sequential()
model.add(Convolution2D(num_filters,kernelS,kernelS, activation='relu', input_shape=(42,42,1)))
model.add(MaxPooling2D(pool_size=(poolS,poolS)))
model.add(Flatten())
model.add(Dropout(.20))
model.add(Dense(dense_neuron, activation='relu'))
model.add(Dropout(.20))
model.add(Dense(len_category, activation='softmax'))

model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
model.fit(trainX, trainY_label,
          batch_size = batch, nb_epoch= epochs, 
          verbose=1,validation_split=0.2)

/home/di0002ya/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), activation="relu", input_shape=(42, 42, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/di0002ya/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Train on 410089 samples, validate on 102523 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor of shape [23104,100] and type float
	 [[{{node training/Adam/zeros_2}} = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [23104,100] values: [0 0 0...]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training/Adam/zeros_2', defined at:
  File "/home/di0002ya/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/di0002ya/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/di0002ya/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/di0002ya/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/di0002ya/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/di0002ya/miniconda3/lib/p

In [44]:
import gc
gc.collect()
del model